# AutoML

In this notebook, Sparkling Water's AutoML is used to see if we can find an optimal model for the [Power Emission](https://archive.ics.uci.edu/ml/datasets/combined+cycle+power+plant) dataset, by automating the training of various types of models

## Library Isolation

We can use [library utilities](https://docs.azuredatabricks.net/user-guide/dev-tools/dbutils.html#dbutils-library) to install Python libraries and create an environment scoped to a notebook session.

Required libraries (PyPi):
* colorama==0.3.8
* h2o-pysparkling-2.4

In [3]:
%python
dbutils.library.installPyPI("colorama", "0.3.8")
dbutils.library.installPyPI("h2o-pysparkling-2.4")

Out[ 4 ]: True

##Import Data

In [5]:
# File location and type
file_location = "/FileStore/tables/powerplant.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ";"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location) \
  .na.drop()

display(df.take(3))

time,AT,V,AP,RH,PE
2017-01-01T00:00:00.000+0000,14.96,41.76,1024.07,73.17,463.26
2017-01-01T01:00:00.000+0000,25.18,62.96,1020.04,59.08,444.37
2017-01-01T02:00:00.000+0000,5.11,39.4,1012.16,92.14,488.56


## Train-Test Split

We want to compare apples to apples here, so we are going to do the same train-test split before we get started with H2O.

In [7]:
(trainDF, testDF) = df.randomSplit([.8, .2], seed=42)
display(trainDF.take(2))

time,AT,V,AP,RH,PE
2017-01-01T00:00:00.000+0000,14.96,41.76,1024.07,73.17,463.26
2017-01-01T04:00:00.000+0000,10.82,37.5,1009.23,96.62,473.9


## Setting up H2O

[H20 documentation in Azure Databricks](https://docs.azuredatabricks.net/spark/latest/mllib/third-party-libraries.html#h2o-sparkling-water).

In [9]:
from pysparkling import *
from pyspark.sql import SparkSession
import h2o
from h2o.automl import H2OAutoML

#Set up H2O Configurations
spark = SparkSession.builder.appName("SparklingWaterApp").getOrCreate()
h2oConf = H2OConf(spark).set("spark.ui.enabled", "false")
hc = H2OContext.getOrCreate(spark, conf=h2oConf)

Connecting to H2O server at http://10.172.249.77:54321 ... successful.
-------------------------- ---------------------------------------------------
H2O cluster uptime: 09 secs
H2O cluster timezone: Etc/UTC
H2O data parsing timezone: UTC
H2O cluster version: 3.24.0.5
H2O cluster version age: 16 days
H2O cluster name: sparkling-water-root_local-1562311079253
H2O cluster total nodes: 1
H2O cluster free memory: 4.489 Gb
H2O cluster total cores: 8
H2O cluster allowed cores: 8
H2O cluster status: accepting new members, healthy
H2O connection url: http://10.172.249.77:54321
H2O connection proxy:
H2O internal security: False
H2O API Extensions: XGBoost, Algos, Amazon S3, AutoML, Core V3, Core V4
Python version: 3.5.2 final
-------------------------- ---------------------------------------------------

Sparkling Water Context:
 * Sparkling Water Version: 2.4.13
 * H2O name: sparkling-water-root_local-1562311079253
 * cluster size: 1
 * list of used nodes:
 (executorId, host, port)
 ------------------------
 (driver,ip-10-172-249-77.us-west-2.compute.internal,54321)
 ------------------------

 Open H2O Flow in browser: http://10.172.249.77:54321 (CMD + click in Mac OSX)

## Convert DF to H2O Frame

In [11]:
trainH2O = hc.as_h2o_frame(trainDF)
testH2O = hc.as_h2o_frame(testDF)

trainH2O[:,0:6].describe()

Rows:7674
Cols:6


 time AT V AP RH PE
------- ------------------- ------------------ ----------------- ------------------ ------------------ ------------------
type time real real real real real
mins 1483228800000.0 1.81 25.36 992.89 25.56 420.26
mean 1500437975628.3528 19.629908782903364 54.26015376596294 1013.2522869429243 73.34368386760464 454.39712275214987
maxs 1517669952000.0 37.11 81.56 1033.3 100.16 495.76
sigma 9943495968.533659 7.4260645085572135 12.70493394481978 5.958514457028822 14.548651793745277 17.030452121090743
zeros 0 0 0 0 0 0
missing 0 0 0 0 0 0
0 2017-01-01 00:00:00 14.96 41.76 1024.07 73.17 463.26
1 2017-01-01 04:00:00 10.82 37.5 1009.23 96.62 473.9
2 2017-01-01 05:00:00 26.27 59.44 1012.23 58.77 443.67
3 2017-01-01 06:00:00 15.89 43.96 1014.02 75.24 467.35
4 2017-01-01 07:00:00 9.48 44.71 1019.12 66.43 478.42
5 2017-01-01 08:00:00 14.64 45.0 1021.78 41.25 475.98
6 2017-01-01 09:00:00 11.74 43.56 1015.14 70.72 477.5
7 2017-01-01 10:00:00 17.99 43.72 1008.64 75.04 453.02
8 2017-01-01 11:00:00 20.14 46.93 1014.66 64.22 453.99
9 2017-01-01 12:00:00 24.34 73.5 1011.31 84.15 440.29

## AutoML

Now we will use AutoML to train the various models. This may take a few minutes (`max_runtime_secs` set to 300)

In [13]:
x = trainH2O.columns
y = "PE"
x.remove(y)
# Defaults to 5 fold cross-val
aml = H2OAutoML(max_runtime_secs = 300, seed = 42, project_name = "Power Emission Prediction", stopping_metric="RMSE", sort_metric="RMSE")
aml.train(x=x, y=y, training_frame=trainH2O)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)
# Note: deep learning model results are not reproducible

AutoML progress: |████████████████████████████████████████████████████████| 100%
 Out[ 8 ]: 
model_id mean_residual_deviance rmse mse mae rmsle
--------------------------------------------------- ------------------------ --------- ----------- --------- ----------
XGBoost_2_AutoML_20190705_075737 11.4068 3.37739 11.4068 2.45056 0.00742745
XGBoost_1_AutoML_20190705_075737 11.4251 3.3801 11.4251 2.46197 0.00743502
StackedEnsemble_AllModels_AutoML_20190705_075737 11.6124 3.4077 11.6124 2.48346 0.00749306
XGBoost_grid_1_AutoML_20190705_075737_model_4 11.8068 3.4361 11.8068 2.512 0.0075548
GBM_4_AutoML_20190705_075737 12.0331 3.46887 12.0331 2.52595 0.0076196
StackedEnsemble_BestOfFamily_AutoML_20190705_075737 12.038 3.46959 12.038 2.54632 0.00762733
XGBoost_grid_1_AutoML_20190705_075737_model_1 12.2231 3.49616 12.2231 2.57028 0.00768736
GBM_grid_1_AutoML_20190705_075737_model_6 12.2498 3.49997 12.2498 2.54773 0.00769531
GBM_3_AutoML_20190705_075737 12.3252 3.51072 12.3252 2.57226 0.00771114
XGBoost_grid_1_AutoML_20190705_075737_model_2 12.3389 3.51268 12.3389 2.57292 0.00772195
GBM_grid_1_AutoML_20190705_075737_model_3 12.5897 3.54819 12.5897 2.60362 0.00779907
GBM_2_AutoML_20190705_075737 12.6152 3.55179 12.6152 2.61194 0.00780578
GBM_1_AutoML_20190705_075737 12.9633 3.60046 12.9633 2.66632 0.00790819
XGBoost_grid_1_AutoML_20190705_075737_model_5 13.8505 3.72163 13.8505 2.79011 0.00816623
XGBoost_3_AutoML_20190705_075737 14.2139 3.77014 14.2139 2.84976 0.00828221
GBM_5_AutoML_20190705_075737 14.4141 3.79659 14.4141 2.8439 0.00833647
GBM_grid_1_AutoML_20190705_075737_model_1 15.1064 3.88669 15.1064 2.92167 0.00853289
XRT_1_AutoML_20190705_075737 16.2127 4.0265 16.2127 3.02687 0.00883347
DRF_1_AutoML_20190705_075737 16.5668 4.07023 16.5668 3.0705 0.00893362
DeepLearning_1_AutoML_20190705_075737 18.3519 4.28392 18.3519 3.33364 0.00940722
GBM_grid_1_AutoML_20190705_075737_model_7 19.6687 4.43494 19.6687 3.45869 0.00969937
GBM_grid_1_AutoML_20190705_075737_model_8 20.3513 4.51124 20.3513 3.54038 0.00987188
DeepLearning_grid_1_AutoML_20190705_075737_model_2 20.3564 4.51181 20.3564 3.54664 0.00990791
GLM_grid_1_AutoML_20190705_075737_model_1 20.7707 4.55749 20.7707 3.62666 0.0100311
DeepLearning_grid_1_AutoML_20190705_075737_model_1 20.9652 4.57878 20.9652 3.60442 0.0100606
DeepLearning_grid_1_AutoML_20190705_075737_model_5 23.7758 4.87604 23.7758 3.81465 0.0107628
GBM_grid_1_AutoML_20190705_075737_model_5 23.8306 4.88166 23.8306 3.58104 0.0106971
DeepLearning_grid_1_AutoML_20190705_075737_model_3 24.8978 4.98977 24.8978 3.86999 0.0109079
DeepLearning_grid_1_AutoML_20190705_075737_model_7 35.7656 5.98044 35.7656 4.81039 0.013137
DeepLearning_grid_1_AutoML_20190705_075737_model_6 111.623 10.5652 111.623 8.78326 0.0235109
XGBoost_grid_1_AutoML_20190705_075737_model_3 121.182 11.0083 121.182 7.63064 0.0245435
GBM_grid_1_AutoML_20190705_075737_model_9 135.615 11.6454 135.615 9.90556 0.0254212
DeepLearning_grid_1_AutoML_20190705_075737_model_4 143.365 11.9735 143.365 10.1988 0.0263287
GBM_grid_1_AutoML_20190705_075737_model_4 278.846 16.6987 278.846 14.4787 0.0365001
GBM_grid_1_AutoML_20190705_075737_model_2 278.881 16.6997 278.881 14.4797 0.0365023
XGBoost_grid_1_AutoML_20190705_075737_model_6 617.186 24.8432 617.186 23.3256 0.0551654
XGBoost_grid_1_AutoML_20190705_075737_model_7 66780.4 258.419 66780.4 258.195 0.836897
XGBoost_grid_1_AutoML_20190705_075737_model_8 186200 431.509 186200 431.2 2.93483

[38 rows x 6 columns]

## Predict Test Data

In [15]:
import pandas as pd
predictPE = aml.predict(testH2O).as_data_frame()
actual = testH2O['PE'].as_data_frame()
predict_vs_actual = pd.concat([predict, actual],axis=1)
predict_vs_actual=spark.createDataFrame(predict_vs_actual)
display(predict_vs_actual.take(10))

predict,PE
445.544677734375,444.37
486.4376525878906,488.56
447.67657470703125,446.48
444.5905456542969,451.28
465.8669128417969,467.54
454.4395751953125,450.69
432.3283081054688,430.12
481.76202392578125,473.62
443.4638366699219,442.99
442.8514709472656,446.22


In [16]:
perf = aml.leader.model_performance(testH2O)
perf

Out[ 20 ]: 
ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 10.364846653760484
RMSE: 3.2194481908799966
MAE: 2.3583327035797232
RMSLE: 0.0070763974216272
Mean Residual Deviance: 10.364846653760484

In [17]:
aml.leader

Out[ 21 ]: Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_2_AutoML_20190705_075737


ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 4.09945106977458
RMSE: 2.024710119936822
MAE: 1.4026386617899624
RMSLE: 0.004462738157887346
Mean Residual Deviance: 4.09945106977458

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 11.406758300617948
RMSE: 3.377389272887854
MAE: 2.4505588073173725
RMSLE: 0.0074274539121886
Mean Residual Deviance: 11.406758300617948
Cross-Validation Metrics Summary: 
 mean sd cv_1_valid cv_2_valid cv_3_valid cv_4_valid cv_5_valid
---------------------- ---------- ----------- ------------ ------------ ------------ ------------ ------------
mae 2.45056 0.0161356 2.42772 2.44603 2.46862 2.42613 2.4843
mean_residual_deviance 11.407 0.619857 10.9831 11.6255 10.8491 10.5613 13.0158
mse 11.407 0.619857 10.9831 11.6255 10.8491 10.5613 13.0158
r2 0.960678 0.00178676 0.961485 0.960434 0.962685 0.962849 0.955937
residual_deviance 11.407 0.619857 10.9831 11.6255 10.8491 10.5613 13.0158
rmse 3.37501 0.0901919 3.31408 3.40962 3.29379 3.24981 3.60774
rmsle 0.00742269 0.000189392 0.00728593 0.00748388 0.00724752 0.00717783 0.00791828
Scoring History: 
 timestamp duration number_of_trees training_rmse training_mae training_deviance
--- ------------------- ---------- ----------------- ------------------ ------------------ -------------------
 2019-07-05 07:58:10 11.998 sec 0.0 454.21646440850117 453.8971227507185 206312.59653975922
 2019-07-05 07:58:10 12.038 sec 5.0 351.5376805805539 351.24305615239945 123578.74086795555
 2019-07-05 07:58:10 12.067 sec 10.0 272.09243545447816 271.8221291469229 74034.29343154936
 2019-07-05 07:58:10 12.095 sec 15.0 210.608850319459 210.3545177381176 44356.08783288429
 2019-07-05 07:58:10 12.123 sec 20.0 163.04454780937422 162.7980167924783 26583.524570363315
--- --- --- --- --- --- ---
 2019-07-05 07:58:13 15.521 sec 155.0 2.1830984330082663 1.5350666896172853 4.765918768203147
 2019-07-05 07:58:14 15.783 sec 160.0 2.131023899536122 1.491288460261007 4.541262860394139
 2019-07-05 07:58:14 16.047 sec 165.0 2.0790226861080487 1.4472231135444154 4.322335329351926
 2019-07-05 07:58:14 16.322 sec 170.0 2.03437653271465 1.4112282743794489 4.138687876860082
 2019-07-05 07:58:14 16.563 sec 172.0 2.024710119936822 1.4026386617899624 4.09945106977458

See the whole table with table.as_data_frame()
Variable Importances: 
variable relative_importance scaled_importance percentage
---------- --------------------- ------------------- ------------
AT 5.7725e+06 1 0.563454
V 3.92046e+06 0.679163 0.382677
AP 343837 0.0595647 0.033562
RH 128995 0.0223464 0.0125912
time 79047 0.0136937 0.00771578

## Save Model

In [19]:
h2o.save_model(aml.leader, path = userhome + "/machine-learning-p/h2o")


Out[ 16 ]: 'dbfs:/user/angelis.vassilis@gmail.com/machine-learning-p/h2o/StackedEnsemble_AllModels_AutoML_20190614_101305'